#Install Packages

In [1]:
!pip install tokenizers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 40.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.1/311.1 kB 36.1 MB/s eta 0:00:00


In [2]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 69.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 106.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 83.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 36.3 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.19.1
    Uninstalling huggingface-hub-0.19.1:
      Successfully uninstalled huggingface-hub-0.19.1
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.15.0
    Uninstalling tokenizers-0.15.0:
      Successfully uninstalled tokenizers-0.15.0


In [3]:
!pip install sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 20.0 MB/s eta 0:00:00


In [4]:
!pip install evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.8 MB/s eta 0:00:00


In [5]:
!pip install datasets

In [6]:
!pip install evaluate

#Import Libraries

In [7]:
import random
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader

from transformers import (
    AdamW,
    T5ForConditionalGeneration,
    T5Tokenizer,
    get_linear_schedule_with_warmup
)

def set_seed(seed):
  random.seed(seed)
  np.random.seed(seed)
  torch.manual_seed(seed)

set_seed(42)

In [8]:
import torch
import json
from tqdm import tqdm
import torch.nn as nn
from torch.optim import Adam
import nltk
import spacy
import string
import evaluate  # Bleu
from torch.utils.data import Dataset, DataLoader, RandomSampler
import pandas as pd
import numpy as np
import transformers
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from transformers import T5Tokenizer, T5Model, T5ForConditionalGeneration, T5TokenizerFast

import warnings
warnings.filterwarnings("ignore")

#Data Preparation

In [9]:
# dataset preparation

from datasets import load_dataset
dataset = load_dataset("amaydle/npc-dialogue")

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/1723 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/192 [00:00<?, ? examples/s]

In [10]:
def prepare_data(data):
  articles = []
  for i in range(len(dataset['train']['Query'])):
    if dataset['train']["Name"][i] in ['Arin', 'Drogath', 'Kaela', 'Lara Croft', 'Thorne', 'Tony Stark']:
      continue
    question = dataset['train']["Name"][i] + ":" + dataset['train']["Biography"][i] + ":" + dataset['train']["Query"][i]
    answer = dataset['train']["Response"][i]
    inputs = {"question": question, "answer": answer}

    articles.append(inputs)

  return articles

In [11]:
data = prepare_data(dataset)

#Model

In [12]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("Salesforce/dialogstudio-t5-base-v1.0")
t5_model = AutoModelForSeq2SeqLM.from_pretrained("Salesforce/dialogstudio-t5-base-v1.0")

input_text = "Answer the following yes/no question by reasoning step-by-step. Can you write 200 words in a single tweet?"
input_ids = tokenizer(input_text, return_tensors="pt").input_ids

outputs = t5_model.generate(input_ids, max_new_tokens=256)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

It is not possible to write 200 words in a tweet. Therefore, the final answer is no.


In [13]:
# optimizer
no_decay = ["bias", "LayerNorm.weight"]
optimizer_grouped_parameters = [
    {
        "params": [p for n, p in t5_model.named_parameters() if not any(nd in n for nd in no_decay)],
        "weight_decay": 0.0,
    },
    {
        "params": [p for n, p in t5_model.named_parameters() if any(nd in n for nd in no_decay)],
        "weight_decay": 0.0,
    },
]
optimizer = AdamW(optimizer_grouped_parameters, lr=3e-4, eps=1e-8)

In [14]:
t5_model.train()

epochs = 10

for epoch in range(epochs):
  print ("epoch ",epoch)
  #for input,output in true_false_adjective_tuples:
  for article in data:

    input = article["question"]
    output = article["answer"]

    input_sent = "answer: "+input+ " </s>"
    ouput_sent = output+" </s>"

    tokenized_inp = tokenizer.encode_plus(input_sent,  max_length=96, pad_to_max_length=True,return_tensors="pt")
    tokenized_output = tokenizer.encode_plus(ouput_sent, max_length=96, pad_to_max_length=True,return_tensors="pt")


    input_ids  = tokenized_inp["input_ids"]
    attention_mask = tokenized_inp["attention_mask"]

    lm_labels= tokenized_output["input_ids"]
    decoder_attention_mask=  tokenized_output["attention_mask"]


    # the forward function automatically creates the correct decoder_input_ids
    output = t5_model(input_ids=input_ids, labels=lm_labels, decoder_attention_mask=decoder_attention_mask, attention_mask=attention_mask)
    loss = output[0]

    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


epoch  0
epoch  1
epoch  2
epoch  3
epoch  4
epoch  5
epoch  6
epoch  7
epoch  8
epoch  9


#Test the Model

In [15]:
def prepare_test_data(data):
    articles = []
    for i in range(len(dataset['test']['Query'])):
      question = dataset['test']["Name"][i] + ":" + dataset['test']["Biography"][i] + ":" + dataset['test']["Query"][i]
      answer = dataset['test']["Response"][i]
      inputs = {"question": question, "answer": answer}

      articles.append(inputs)

    return articles

In [16]:
test_data = prepare_test_data(dataset)

In [17]:
def test_model(test_data, t5_model):

  i = 0
  ref_answers = []
  predicted_answers = []
  answers = {}
  for test_data_item in test_data:

    test_sent = test_data_item['question']

    test_tokenized = tokenizer.encode_plus(test_sent, return_tensors="pt")

    test_input_ids  = test_tokenized["input_ids"]
    test_attention_mask = test_tokenized["attention_mask"]

    t5_model.eval()
    beam_outputs = t5_model.generate(
      input_ids=test_input_ids,attention_mask=test_attention_mask,
      max_length=64,
      early_stopping=True,
      num_beams=10,
      num_return_sequences=3,
      no_repeat_ngram_size=2
    )

    #print("Question: "+test_data_item['question'])
    ref_answers.append(test_data_item['answer'])

    for beam_output in beam_outputs:
      sent = tokenizer.decode(beam_output, skip_special_tokens=True,clean_up_tokenization_spaces=True)
      predicted_answers.append(sent)
      # Just pick the first answer
      break
  return ref_answers, predicted_answers

ref, pred = test_model(test_data, t5_model)

In [18]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"
!pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=f766ffa026d4ea58e30a73fa251cc8d378258be85c072533e12bed355b9fb916
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [19]:
import evaluate
rouge = evaluate.load('rouge')
results = rouge.compute(predictions=pred, references=ref)
print(results)

{'rouge1': 0.2222648343675242, 'rouge2': 0.0609602556407676, 'rougeL': 0.19972490851407798, 'rougeLsum': 0.20016791656072364}


In [36]:
import re

name = []
biography = []
question = []

for test_data_item in test_data:
    test_sent = test_data_item['question']
    match = re.search("^(.*):(.*):(.*)", test_sent)
    name.append(match.group(1))
    biography.append(match.group(2))
    question.append(match.group(3))

In [37]:
from pandas import DataFrame

df = DataFrame({'Name': name, 'Question': question, 'Ref_response': ref, 'Pred_response': pred, 'Biography': biography})

In [38]:
df.head(5)

,Name,Question,Ref_response,Pred_response,Biography
0,Naina Mathur,What is the biggest challenge you face as a te...,Ensuring every student receives the individual...,Keeping my students focused and ensuring that ...,Naina Mathur is a determined and passionate te...
1,Zephyr,What motivates you to play pranks on people?,"It's just who I am, I guess. I love seeing peo...",The thought of what could be accomplished if I...,Zephyr is a mischievous fairy who loves playin...
2,"Arn, the Knight Templar",Can you describe yourself in three words?,"""Courageous, dedicated, honorable.""","""Quiet, reserved, loyal.""","Arn is a highly skilled and honorable knight,"
3,Arinthal,Have you ever been to a city?,Cities are noisy and overwhelming.,"Yes, I have been to many cities in my travels.",Arinthal is an elven ranger from the ancient f...
4,Tiger,What is the most valuable thing in your life?,My country and the people I love.,My most valuable possession is my country's go...,Tiger is a highly skilled and fearless spy wor...


In [39]:
df.to_excel('DialogStudio_T5_fewshot.xlsx', sheet_name='sheet1', index=False)

#Save Model

In [21]:
t5_model.save_pretrained("/content/sample_data/T5_dialogstudio_fewshot_model", from_pt=True)

In [22]:
#Deprecated in favor of model push to huggingface

!tar -czvf model.tar.gz /content/sample_data/T5_dialogstudio_fewshot_model

tar: Removing leading `/' from member names
/content/sample_data/T5_dialogstudio_fewshot_model/
/content/sample_data/T5_dialogstudio_fewshot_model/config.json
/content/sample_data/T5_dialogstudio_fewshot_model/generation_config.json
/content/sample_data/T5_dialogstudio_fewshot_model/model.safetensors


In [24]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|
    
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) n
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [27]:
t5_model.push_to_hub("finetuned-t5-dialogstudio-npc")
tokenizer.push_to_hub("finetuned-t5-dialogstudio-npc")

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Himabindu/finetuned-t5-dialogstudio-npc/commit/71ba04445ad6088052b61bfe242af48914facdf8', commit_message='Upload tokenizer', commit_description='', oid='71ba04445ad6088052b61bfe242af48914facdf8', pr_url=None, pr_revision=None, pr_num=None)